In [2]:
# Lesson7：Backtrader来啦：可视化篇（重构）
# link: https://mp.weixin.qq.com/s/WA7Dgr_kcZz-WhriHkf4AQ

# =============================================================================
#%%
# 第1章 observers 观测器
'''
backtrader.observers.Broker：
    记录了经纪商 broker 中各时间点的可用资金和总资产；可视化时，会同时展示 cash 和 values 曲线；
    如果想各自单独展示 cash 和 values，可以分别调用 backtrader.observers.Cash 和 backtrader.observers.Value；

backtrader.observers.BuySell：
    记录了回测过程中的买入和卖出信号；可视化时，会在价格曲线上标注买卖点；

backtrader.observers.Trades：
    记录了回测过程中每次交易的盈亏（从买入建仓到卖出清仓算一次交易）；可视化时，会绘制盈亏点；

backtrader.observers.TimeReturn：
    记录了回测过程中的收益序列；可视化时，会绘制 TimeReturn 收益曲线；

backtrader.observers.DrawDown：
    记录了回测过程的回撤序列；可视化时，绘制回撤曲线；

backtrader.observers.Benchmark：
    记录了业绩基准的收益序列，业绩基准的数据必须事先通过 adddata、resampledata、replaydata 等数据
    添加函数添加进大脑中 cerebro；可视化时，会同时绘制策略本身的收益序列
    （即：backtrader.observers.TimeReturn 绘制的收益曲线）和业绩基准的收益曲线。

'''
#%%
## 第1.1节 如何添加 observers
'''
addobserver(obscls, *args, **kwargs):
    observers 观测器是通过 addobserver() 添加给大脑 cerebro 的：
    参数 obscls 对应 observers 类下的观测器、*args, **kwargs 对应观测器支持设置的参数

bt.Cerebro(stdstats=False):
    对于 Broker、Trades、BuySell 3个观测器，默认是自动添加给 cerebro 的，
    可以在实例化大脑时，通过 stdstats 来控制：bt.Cerebro(stdstats=False) 表示可视化时，
    不展示 Broker、Trades、BuySell 观测器；反之，自动展示；默认情况下是自动展示。
'''
import backtrader as bt

cerebro = bt.Cerebro(stdstats=False)
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.TimeReturn)
# 添加业绩基准时，需要事先将业绩基准的数据添加给 cerebro
banchdata = bt.feeds.PandasData(dataname=data, fromdate=st_date, todate=ed_date)
cerebro.adddata(banchdata, name='xxxx')
cerebro.addobserver(bt.observers.Benchmark, data=banchdata)

#%%
## 第1.2节 如何读取 observers 中的数据
'''
observers 中记录了各种回测数据，可以将其看作是一个支持可视化展示的数据存储器，
所以 observers 属于 lines 对象。如果想在 Strategy 中读取 observers 中的数据，
就会用到 line 的相关操作，observers 的数据通过 self.stats 对象 来连接。

【注意时间】
observers 是在所有指标被计算完之后、在执行 Strategy 的 next 方法之后才运行并统计数据的，
所以读取的最新数据 [0] 相对与 next 的当前时刻是晚一天的。
比如 2019-04-08 的总资产为 99653.196672，2019-04-09 的总资产为 99599.008652，
2019-04-09 这一天的收益为 -0.0005437，如果在 next 通过 self.stats.timereturn.line[0] 提取，
取值为 -0.0005437 时，对应的 next 的当前时间是  2019-04-10
'''
class MyStrategy(bt.Strategy):
    def next(self):
        # 当前时点的前一天的可用现金
        self.stats.broker.cash[0]
        # 当前时点的前一天的总资产
        self.stats.broker.value[0]
        # 获取当前时刻前一天的收益
        self.stats.timereturn.line[0]
        # observers 取得[0]值，对应的 next 中 self.data.datetime[-1] 这一天的值

'''
如果想要将 observers  中的数据保存到本地，可以通过 writer  写入本地文件，如下面的读写到本地 CSV 文件：
'''
import csv

class TestStrategy(bt.Strategy):
    ...
    def start(self):
        self.mystats = csv.writer(open("mystats.csv", "w"))
        self.mystats.writerow(['datetime',
                               'drawdown', 'maxdrawdown',
                               'timereturn',
                               'value', 'cash'])
    def next(self): 
        self.mystats.writerow([self.data.datetime.date(-1).strftime('%Y-%m-%d'),
                               '%.4f' % self.stats.drawdown.drawdown[0],
                               '%.4f' % self.stats.drawdown.maxdrawdown[0],
                               '%.4f' % self.stats.timereturn.line[0],
                               '%.4f' % self.stats.broker.value[0],
                               '%.4f' % self.stats.broker.cash[0]])
    def stop(self):  
        self.mystats.writerow([self.data.datetime.date(0).strftime('%Y-%m-%d'),
                               '%.4f' % self.stats.drawdown.drawdown[0],
                               '%.4f' % self.stats.drawdown.maxdrawdown[0],
                               '%.4f' % self.stats.broker.value[0],
                               '%.4f' % self.stats.broker.cash[0]])
        
    # 当运行到最后一根 bar 后， next 中记录的是上一根 bar 的收益
    # stop 是在 next 运行完后才运行的，此时 observers 已经计算完 最后一根 bar 的收益了
    # 所以可以在 stop 中获取最后一根 bar 的收益

#%%
## 第1.3节 自定义 observers 
'''
和之前各种自定义一致，自定义 observers 同样是在继承父类  bt.observer.Observer 的基础上，自定义新的的observers。
下面是 Backtrader 官网提供的例子，用于统计已成功创建的订单的价格和到期订单的价格。
'''
class OrderObserver(bt.observer.Observer):
    lines = ('created', 'expired',)

    plotinfo = dict(
                     plot=True, subplot=True, plotlinelabels=True
                    )

    plotlines = dict(
                     created=dict(marker='*', markersize=8.0, color='lime', fillstyle='full'),
                     expired=dict(marker='s', markersize=8.0, color='red', fillstyle='full')
                    )

    def next(self):
        for order in self._owner._orderspending:
            if order.data is not self.data:
                continue

            if not order.isbuy():
                continue

            # Only interested in "buy" orders, because the sell orders
            # in the strategy are Market orders and will be immediately
            # executed

            if order.status in [bt.Order.Accepted, bt.Order.Submitted]:
                self.lines.created[0] = order.created.price

            elif order.status in [bt.Order.Expired]:
                self.lines.expired[0] = order.created.price

'''
observers 本身是 Lines 对象，所以构建逻辑与自定义 Indicator 类似，
将要统计的数据指定为相应的 line，然后随着回测的进行依次存入数据；

作为 Lines 对象的 Observers 和 Indicator ，类内部都有 
plotinfo = dict(...)、plotlines = dict(...) 属性，
用于回测结束后通过 cerebro.plot() 方法进行可视化展示；

有时候如果想修改 Backtrader 已有观测器的相关属性，可以直接继承该观测器，然后设置属性取值进行修改。
如下面在原始 bt.observers.BuySell 的基础上，修改买卖点的样式。
'''
class my_BuySell(bt.observers.BuySell):
    params = (('barplot', True), ('bardist', 0.02),)
    # 将 三角形改为箭头
    plotlines = dict(
                     buy=dict(marker=r'$\Uparrow$', markersize=10.0, color='#d62728' ),
                     sell=dict(marker=r'$\Downarrow$', markersize=10.0, color='#2ca02c')
                    )
    
# =============================================================================
#%%
# 第2章 plot() 图形绘制

'''
cerebro.plot() 支持回测如下 3 大内容：
    
    Data Feeds：即在回测开始前，通过 adddata、replaydata、resampledata 等方法导入大脑的原始数据；
    Indicators ：即回测时构建的各类指标，比如在 strategy 中构建的指标、通过 addindicator 添加的；
    Observers ：即上文介绍的观测器对象；

在绘制图形时，默认是将 Data Feeds 绘制在主图上；
Indicators 有的与 Data Feeds 一起绘制在主图上，比如均线，
有的以子图形式绘制；Observers 通常绘制在子图上。
'''

## 第2.1节 plot() 中的参数
plot(plotter=None, # 包含各种绘图属性的对象或类，如果为None，默认取 PlotScheme 类，如下所示
     numfigs=1,    # 是否将图形拆分成多幅图展示，如果时间区间比较长，建议分多幅展示
     iplot=True,   # 在 Jupyter Notebook 上绘图时是否自动 plot inline
     **kwargs)     # 对应 PlotScheme 中的各个参数

# PlotScheme 中的参数如下所示
class PlotScheme(object):
    def __init__(self):
        # X轴、Y轴同时使用紧密排列视图
        self.ytight = False

        # 副图Y轴距离顶和底的边距，不会覆盖plotinfo.plotymargin的设置
        self.yadjust = 0.0
        # 每条新的线都叠加到上一根线下面，如果为False则覆盖到上一根线上面
        self.zdown = True
        # X轴日期的旋转角度
        self.tickrotation = 15

        # 主图中（数据）被分割的数量，与子图的数量成正比
        self.rowsmajor = 5

        # 副图中（指标、观察器）被分割的数量，与子图的数量成正比
        # 与rowsmajor共同决定了主图和副图之间的比例
        self.rowsminor = 1

        # 子图间距
        self.plotdist = 0.0

        # 所有图表添加网格线
        self.grid = True

        # 默认的股价线绘图方式：line（收盘价线），可以另选：'bar'、'candle'
        self.style = 'line'

        # line（收盘价线）颜色
        self.loc = 'black'
        # bar/candle上涨线的颜色（灰度：0.75）
        self.barup = '0.75'
        # bar/candle下跌线的颜色（红色）
        self.bardown = 'red'
        # bar/candle的透明度（1表示完全不透明）
        self.bartrans = 1.0

        # 上涨/下跌线：True=填充，False：透明
        self.barupfill = True
        self.bardownfill = True
        # 填充颜色的透明度
        self.fillalpha = 0.20

        # 是否画出成交量，如果输入数据没有成交量，即使设置为True也不会画出
        self.volume = True

        # 是否直接在股价图上画成交量(True)，还是在副图中画成交量(False)
        self.voloverlay = True
        # 如果voloverlay = True，设置成交量绘图的大小
        self.volscaling = 0.33
        # 如果voloverlay = True，一旦成交量图覆盖股价线太多的话，
        # 设置volpushup移动股价以便让人看清全图
        self.volpushup = 0.00

        # 上涨时成交量柱的颜色
        self.volup = '#aaaaaa'
        # 下跌时成交量柱的颜色
        self.voldown = '#cc6073'
        # 设置当成交量图覆盖住股价线时的透明度
        self.voltrans = 0.50

        # 文字标签的透明度（当前未使用）
        self.subtxttrans = 0.66
        # 标签的文字大小
        self.subtxtsize = 9

        # 图例的透明度（当前未使用）
        self.legendtrans = 0.25
        # 设置指标的图例显示
        self.legendind = True
        # 设置指标的图例位置
        self.legendindloc = 'upper left'

        # 是否显示股价线、指标线的最后一个数据值
        self.linevalues = True

        # 是否在股价线、指标线的最后一个数据打标签
        self.valuetags = True

        # 水平线的默认颜色（0.66灰色） (参考plotinfo.plothlines)
        self.hlinescolor = '0.66'  # shade of gray
        # 水平线的默认样式
        self.hlinesstyle = '--'
        # 水平线的默认宽度
        self.hlineswidth = 1.0

        # 颜色主题：Tableau 10【如果想修改主题色，需要重新定义 tableau10 变量】
        self.lcolors = tableau10

        # X轴显示tick时间的strftime字符串格式
        self.fmt_x_ticks = None

        # X轴显示数据值的strftime字符串格式
        self.fmt_x_data = None

'''
如果想要系统性修改图形样式，可以重新定义 PlotScheme 类，然后修改里面用到的参数；
也可以直接在plot() 中修改。
'''

# 通过参数形式来设置
cerebro.plot(iplot=False,
             style='candel',  # 设置主图行情数据的样式为蜡烛图
             lcolors=colors , # 重新设置主题颜色
             plotdist=0.1,    # 设置图形之间的间距
             barup = '#ff9896', bardown='#98df8a', # 设置蜡烛图上涨和下跌的颜色
             volup='#ff9896', voldown='#98df8a', # 设置成交量在行情上涨和下跌情况下的颜色
             ....)

'''
关于主题颜色，Backtrader 提供了Tableau 10 、Tableau 10 Light、Tableau 20 3种主题色，默认是以 Tableau 10 为主题色。
但是看源代码，不知道如何修改 lcolors，源码 scheme.py 文件中的 tableau10 只一个变量，直接赋值给 self.lcolors = tableau10，
如果在我们在自己的的 notebook上运行 lcolors=tableau10 会报错，提示 tableau10 变量不存在。
所以，如果想修改主题色，需要重新定义 tableau10 变量。

从源码中复制的主题色，后面都注释了索引号，而 Backtrader 在绘制图形时，选择颜色的顺序依次是这样的：
tab10_index = [3, 0, 2, 1, 2, 4, 5, 6, 7, 8, 9]；
tab10_index 中的序号对应的是 主题色 的索引号；

每一幅图，依次取 tab10_index 中的序号对应的颜色来绘制，
比如 MACD 有 3 条 line：
line0 的颜色为 tab10_index[0] = 3，也就是 lcolors=tableau10 中的索引号为 3 对应的颜色 'crimson'；
line1 的颜色为 tab10_index[1] = 0，也就是 lcolors=tableau10 中的索引号为 0 对应的颜色 'blue'；
'''
# 定义主题色变量：直接从源码 scheme.py 中复制的
tableau20 = [
            'steelblue', # 0
            'lightsteelblue', # 1
            'darkorange', # 2
            'peachpuff', # 3
            'green', # 4
            'lightgreen', # 5
            'crimson', # 6
            'lightcoral', # 7
            'mediumpurple', # 8
            'thistle', # 9
            'saddlebrown', # 10
            'rosybrown', # 11
            'orchid', # 12
            'lightpink', # 13
            'gray', # 14
            'lightgray', # 15
            'olive', # 16
            'palegoldenrod', # 17
            'mediumturquoise', # 18
            'paleturquoise', # 19
            ]

tableau10 = [
            'blue', # 'steelblue', # 0
            'darkorange', # 1
            'green', # 2
            'crimson', # 3
            'mediumpurple', # 4
            'saddlebrown', # 5
            'orchid', # 6
            'gray', # 7
            'olive', # 8
            'mediumturquoise', # 9
            ]

tableau10_light = [
                    'lightsteelblue', # 0
                    'peachpuff', # 1
                    'lightgreen', # 2
                    'lightcoral', # 3
                    'thistle', # 4
                    'rosybrown', # 5
                    'lightpink', # 6
                    'lightgray', # 7
                    'palegoldenrod', # 8
                    'paleturquoise', # 9
                  ]

# 选一个主题色做修改
cerebro.plot(lcolors=tableau10)


# 当然也可以选自己喜欢的主题色
mycolors = ['#729ece', '#ff9e4a', '#67bf5c',
           '#ed665d', '#ad8bc9', '#a8786e',
           '#ed97ca', '#a2a2a2', '#cdcc5d', '#6dccda']

cerebro.plot(lcolors=mycolors)

#%%
## 第2.2节 局部绘图参数设置
'''
对于 Indicators  和 Observers 的可视化设置，通过类内部的 plotinfo = dict(...)、plotlines = dict(...) 属性来控制，
其中 plotinfo 主要对图形整体布局进行设置，plotlines 主要对具体 line 的样式进行设置。
'''
plotinfo = dict(plot=True, # 是否绘制
                subplot=True, # 是否绘制成子图
                plotname='', # 图形名称
                plotabove=False, # 子图是否绘制在主图的上方
                plotlinelabels=False, # 主图上曲线的名称
                plotlinevalues=True, # 是否展示曲线最后一个时间点上的取值
                plotvaluetags=True, # 是否以卡片的形式在曲线末尾展示最后一个时间点上的取值
                plotymargin=0.0, # 用于设置子图 y 轴的边界
                plothlines=[a,b,...], # 用于绘制取值为 a,b,... 的水平线
                plotyticks=[], # 用于绘制取值为 a,b,... y轴刻度
                plotyhlines=[a,b,...], # 优先级高于plothlines、plotyticks，是这两者的结合
                plotforce=False, # 是否强制绘图
                plotmaster=None, # 用于指定主图绘制的主数据
                plotylimited=True,
                # 用于设置主图的 y 轴边界，
                # 如果True，边界只由主数据 data feeds决定，无法完整显示超出界限的辅助指标；
                # 如果False, 边界由主数据 data feeds和指标共同决定，能确保所有数据都能完整展示
                )

# 修改交易观测器的样式
class my_Trades(bt.observers.Trades):
    plotlines = dict(
                    pnlplus = dict(_name='Positive',
                                    marker='^', color='#ed665d',
                                    markersize=8.0, fillstyle='full'),

                    pnlminus = dict(_name='Negative',
                                    marker='v', color='#729ece',
                                    markersize=8.0, fillstyle='full')
                    )
    
# 修改买卖点样式
class my_BuySell(bt.observers.BuySell):
    params = (('barplot', True), ('bardist', 0.02)) # bardist 控制买卖点与行情线之间的距离
    plotlines = dict(
                    buy=dict(marker=r'$\Uparrow$', markersize=10.0, color='#d62728' ),
                    sell=dict(marker=r'$\Downarrow$', markersize=10.0, color='#2ca02c')
                    )
#%%
## 第2.3节 部分修改效果
'''
一般主图的样式通过 plot() 中的参数来设置；
Indicators  和 Observers 的样式通过继承原始类，然后通过修改plotinfo 和 plotlines 属性来设置；
'''

'【蜡烛图样式】'
import matplotlib.pyplot as plt

plt.style.use('seaborn') # 使用 seaborn 主题
plt.rcParams['figure.figsize'] = 20, 10  # 全局修改图大小

# 修改 Trades 观测器的样式
class my_Trades(bt.observers.Trades):
    plotlines = dict(
                        pnlplus = dict(_name='Positive',
                                       marker='^', color='#ed665d',
                                       markersize=8.0, fillstyle='full'),

                        pnlminus = dict(_name='Negative',
                                        marker='v', color='#729ece',
                                        markersize=8.0, fillstyle='full')
                    )

# 修改 BuySell 的样式
class my_BuySell(bt.observers.BuySell):
    params = (('barplot', True), ('bardist', 0.02))
    plotlines = dict(
                    buy=dict(marker=r'$\Uparrow$', markersize=10.0, color='#d62728' ),
                    sell=dict(marker=r'$\Downarrow$', markersize=10.0, color='#2ca02c')
                    )
    
    
# 简单均线策略
class TestStrategy(bt.Strategy):
    ......
    
# 绘制图形
cerebro1 = bt.Cerebro(stdstats=False)
......
# 添加观测器
cerebro1.addobserver(bt.observers.DrawDown)
cerebro1.addobserver(bt.observers.Benchmark, data=datafeed1)
cerebro1.addobserver(bt.observers.Broker)
cerebro1.addobserver(my_Trades)
cerebro1.addobserver(my_BuySell)
#先运行策略
rasult = cerebro1.run()
#策略运行完后，绘制图形
colors = ['#729ece', '#ff9e4a', '#67bf5c', '#ed665d', '#ad8bc9', '#a8786e', '#ed97ca', '#a2a2a2', '#cdcc5d', '#6dccda']
tab10_index = [3, 0, 2, 1, 2, 4, 5, 6, 7, 8, 9]
cerebro1.plot(iplot=False,
              style='line', # 绘制线型价格走势，可改为 'candel' 样式
              lcolors=colors,
              plotdist=0.1,
              bartrans=0.2,
              volup='#ff9896',
              voldown='#98df8a',
              loc='#5f5a41',
              # 蜡烛之间会比较拥挤，可以通过设置 numfigs=2，分 2 部分绘制
              # numfigs=2,
              grid=False) # 删除水平网格

# =============================================================================
#%%
# 第3章 基于收益序列进行可视化
'''
Backtrader 自带的绘图工具方便好用，不过平时在汇报策略回测结果时，可能更关注的是策略的累计收益曲线和业绩评价指标等结果，
而这些回测统计信息只需基于回测返回的 TimeReturn 收益序列做简单计算即可得到。
下面是基于 Backtrader 回测返回的分析器 TimeReturn、pyfolio、matplotlib 得到的可视化图形。
'''
import numpy as np
import pandas as pd
import backtrader as bt
import warnings
warnings.filterwarnings('ignore')

import tushare as ts
import json
with open(r'Data/tushare_token.json','r') as load_json:
    token_json = json.load(load_json)
token = token_json['token']
ts.set_token(token) 
pro = ts.pro_api(token)

# 使用Tushare获取数据，要严格保持OHLC的格式
df = ts.pro_bar(ts_code='600276.SH', adj='qfq',start_date='20160101', end_date='20211015')
df = df[['trade_date', 'open', 'high', 'low', 'close','vol']]
df.columns = ['trade_date', 'open', 'high', 'low', 'close','volume']
df.trade_date = pd.to_datetime(df.trade_date)
# 索引必须是日期
df.index = df.trade_date
# 日期必须要升序
df.sort_index(inplace=True)

# Create a Stratey
class TestStrategy(bt.Strategy):
    # 设定参数，便于修改
    params = (
        ('maperiod', 60),
    )

    def log(self, txt, dt=None):
        ''' 
        日志函数：打印结果
        datas[0]：传入的数据，包含日期、OHLC等数据
        datas[0].datetime.date(0)：调用传入数据中的日期列
        '''
        dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # dataclose变量：跟踪当前收盘价
        self.dataclose = self.datas[0].close

        # order变量：跟踪订单状态
        self.order = None
        # buyprice变量：买入价格
        self.buyprice = None
        # buycomm变量：买入时佣金费用
        self.buycomm = None

        # 指标：简单移动平均 MovingAverageSimple【15天】
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0],period=self.params.maperiod)
        
        # 添加画图专用指标
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)

        self.my_indicator_kdj = bt.indicators.StochasticFull(self.datas[0],period=9)
        
    def notify_order(self, order):
        '''订单状态通知(order.status)：提示成交状态'''
        if order.status in [order.Submitted, order.Accepted]:
            # 如果订单只是提交状态，那么啥也不提示
            return

        # 检查订单是否执行完毕
        # 注意：如果剩余现金不足，则会被拒绝！
        if order.status in [order.Completed]:
            if order.isbuy():
                # 买入信号记录：买入价、买入费用、买入佣金费用
                self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                # 卖出信号记录：卖出价、卖出费用、卖出佣金费用
                self.log(
                        'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
            
            # 记录订单执行的价格柱的编号（即长度）
            self.bar_executed = len(self)

        # 如果订单被取消/保证金不足/被拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 如果没有查询到订单，则订单为None
        self.order = None

    def notify_trade(self, trade):
        '''交易状态通知：查看交易毛/净利润'''
        if not trade.isclosed:
            return
        # 交易毛利润：trade.pnl、交易净利润：trade.pnlcomm（扣除佣金）
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                    (trade.pnl, trade.pnlcomm))

    # 交易策略主函数
    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 检查一下是否有订单被挂起，如果有的话就先不下单
        if self.order:
            return
        
        # 检查一下目前是否持有头寸，如果没有就建仓
        if not self.position:

            # 如果 K角度>0 且 K>D 且  K<50:
            if (self.my_indicator_kdj.percK[0] > self.my_indicator_kdj.percK[-1]) & (self.my_indicator_kdj.percK[0] > self.my_indicator_kdj.percD[0]) & (self.my_indicator_kdj.percK[0]<20):
                # 买买买！先记录一下买入价格（收盘价）
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：buy()：开仓买入，买入价是下一个数据，即【开盘价】
                self.order = self.buy()
        else:
             # 如果已经建仓，并持有头寸，则执行卖出指令
             # 如果 K角度<0 且 K<D 且 K>70:
            if (self.my_indicator_kdj.percK[0] < self.my_indicator_kdj.percK[-1]) & (self.my_indicator_kdj.percK[0] < self.my_indicator_kdj.percD[0]):
                # 卖!卖!卖!
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：sell()：平仓卖出，卖出价是下一个数据，即【开盘价】
                self.order = self.sell()

# 创建实例
cerebro = bt.Cerebro()
 # 添加策略
cerebro.addstrategy(TestStrategy)
# 添加数据源
data = bt.feeds.PandasData(dataname=df)
# 输入数据源
cerebro.adddata(data)
# 设置初始现金：10万
cerebro.broker.setcash(1000000.0)
# 设定每次买入的股票数量：10股
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
# 设置佣金费率：双边0.1%
cerebro.broker.setcommission(commission=0.001)


# 回测时需要添加 TimeReturn 分析器
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
result = cerebro.run()

# 提取收益序列
pnl = pd.Series(result[0].analyzers._TimeReturn.get_analysis())
# 计算累计收益
cumulative = (pnl + 1).cumprod()
# 计算回撤序列
max_return = cumulative.cummax()
drawdown = (cumulative - max_return) / max_return

# 计算收益评价指标
import pyfolio as pf
# 按年统计收益指标
perf_stats_year = (pnl).groupby(pnl.index.to_period('y')).apply(lambda data: pf.timeseries.perf_stats(data)).unstack()
# 统计所有时间段的收益指标
perf_stats_all = pf.timeseries.perf_stats((pnl)).to_frame(name='all')
perf_stats = pd.concat([perf_stats_year, perf_stats_all.T], axis=0)
perf_stats_ = round(perf_stats,4).reset_index()


# 绘制图形
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import matplotlib.ticker as ticker # 导入设置坐标轴的模块
# plt.style.use('seaborn') 
plt.style.use('dark_background')

fig, (ax0, ax1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[1.5, 4]}, figsize=(20,8))
cols_names = ['date', 'Annual\nreturn', 'Cumulative\nreturns', 'Annual\nvolatility',
             'Sharpe\nratio', 'Calmar\nratio', 'Stability', 'Max\ndrawdown',
             'Omega\nratio', 'Sortino\nratio', 'Skew', 'Kurtosis', 'Tail\nratio',
             'Daily value\nat risk']

# 绘制表格
ax0.set_axis_off() # 除去坐标轴
table = ax0.table(cellText = perf_stats_.values,
                bbox=(0,0,1,1), # 设置表格位置， (x0, y0, width, height)
                rowLoc = 'right', # 行标题居中
                cellLoc='right' ,
                colLabels = cols_names, # 设置列标题
                colLoc = 'right', # 列标题居中
                edges = 'open' # 不显示表格边框
                )
table.set_fontsize(13)

# 绘制累计收益曲线
ax2 = ax1.twinx()
ax1.yaxis.set_ticks_position('right') # 将回撤曲线的 y 轴移至右侧
ax2.yaxis.set_ticks_position('left') # 将累计收益曲线的 y 轴移至左侧
# 绘制回撤曲线
drawdown.plot.area(ax=ax1, label='drawdown (right)', rot=0, alpha=0.3, fontsize=13, grid=False)
# 绘制累计收益曲线
(cumulative).plot(ax=ax2, color='#F1C40F' , lw=3.0, label='cumret (left)', rot=0, fontsize=13, grid=False)
# 不然 x 轴留有空白
ax2.set_xbound(lower=cumulative.index.min(), upper=cumulative.index.max())
# 主轴定位器：每 5 个月显示一个日期：根据具体天数来做排版
ax2.xaxis.set_major_locator(ticker.MultipleLocator(100))
# 同时绘制双轴的图例
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
plt.legend(h1+h2,l1+l2, fontsize=12, loc='upper left', ncol=1)

fig.tight_layout() # 规整排版
plt.show()

SyntaxError: invalid syntax (1510369704.py, line 295)

In [3]:
import backtrader as bt
import backtrader as bt
import pandas as pd
import datetime

# 回测策略
class StockSelectStrategy(bt.Strategy):
    '''多因子选股 - 基于调仓表'''
    def __init__(self):
        # 读取调仓表，表结构如下所示：
        # trade_date sec_code weight
        # 0 2019-01-31 000006.SZ 0.007282
        # 1 2019-01-31 000008.SZ 0.009783
        # ... ... ... ...
        # 2494 2021-01-28 688088.SH 0.007600
        self.buy_stock = pd.read_csv("Data/trade_info.csv", parse_dates=['trade_date'])
        # 读取调仓日期，即每月的最后一个交易日，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        self.trade_dates = pd.to_datetime(self.buy_stock['trade_date'].unique()).tolist()
        self.order_list = [] # 记录以往订单，方便调仓日对未完成订单做处理
        self.buy_stocks_pre = [] # 记录上一期持仓
    

    def log(self, txt, dt=None):
        ''' 策略日志打印函数'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def next(self):
        dt = self.datas[0].datetime.date(0) # 获取当前的回测时间点
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            # 在调仓之前，取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                for od in self.order_list:
                    self.cancel(od) # 如果订单未完成，则撤销订单
                self.order_list = [] #重置订单列表
            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list) # 打印持仓列表
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock) # 打印平仓列表
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data)
                        self.order_list.append(od) # 记录卖出订单
            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)
       
            self.buy_stocks_pre = long_list # 保存此次调仓的股票列表
        
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已经处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                        'BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref, # 订单编号
                         order.executed.price, # 成交价
                         order.executed.value, # 成交额
                         order.executed.comm, # 佣金
                         order.executed.size, # 成交量
                         order.data._name)) # 股票名称
            else: # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                            (order.ref,
                             order.executed.price,
                             order.executed.value,
                             order.executed.comm,
                             order.executed.size,
                             order.data._name))
        

# 实例化 cerebro
cerebro = bt.Cerebro()
# 读取行情数据
daily_price = pd.read_csv("Data/daily_price.csv", parse_dates=['datetime'])
daily_price = daily_price.set_index(['datetime']) # 将datetime设置成index
# 按股票代码，依次循环传入数据
for stock in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique()) # 获取回测区间内所有交易日
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    # 导入数据
    datafeed = bt.feeds.PandasData(dataname=data_,
                                   fromdate=datetime.datetime(2019,1,2),
                                   todate=datetime.datetime(2021,1,28))
    cerebro.adddata(datafeed, name=stock) # 通过 name 实现数据集与股票的一一对应
    print(f"{stock} Done !")
# 初始资金 100,000,000
cerebro.broker.setcash(100000000.0)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)
# 将编写的策略添加给大脑，别忘了 ！
cerebro.addstrategy(StockSelectStrategy)
# 回测时需要添加 PyFolio 分析器
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
result = cerebro.run()
# 借助 pyfolio 进一步做回测结果分析
pyfolio = result[0].analyzers.pyfolio # 注意：后面不要调用 .get_analysis() 方法
# 或者是 result[0].analyzers.getbyname('pyfolio')

returns, positions, transactions, gross_lev = pyfolio.get_pf_items()

import pyfolio as pf
returns, positions, transactions, gross_lev = pyfolio.get_pf_items()


pf.create_full_tear_sheet(returns)

600466.SH Done !
603228.SH Done !
600315.SH Done !
000750.SZ Done !
002588.SZ Done !
002926.SZ Done !
603816.SH Done !
002517.SZ Done !
600366.SH Done !
001914.SZ Done !
000732.SZ Done !
600733.SH Done !
000930.SZ Done !
002093.SZ Done !
603056.SH Done !
002078.SZ Done !
600978.SH Done !
600329.SH Done !
601872.SH Done !
600058.SH Done !
601019.SH Done !
600497.SH Done !
002563.SZ Done !
600699.SH Done !
601608.SH Done !
002051.SZ Done !
002603.SZ Done !
000636.SZ Done !
000980.SZ Done !
002217.SZ Done !
600291.SH Done !
600827.SH Done !
603369.SH Done !
000829.SZ Done !
002317.SZ Done !
002509.SZ Done !
002557.SZ Done !
002212.SZ Done !
000415.SZ Done !
000860.SZ Done !
600317.SH Done !
600060.SH Done !
600500.SH Done !
300474.SZ Done !
600273.SH Done !
000681.SZ Done !
603707.SH Done !
600967.SH Done !
600415.SH Done !
002056.SZ Done !
600267.SH Done !
600161.SH Done !
002670.SZ Done !
002127.SZ Done !
002815.SZ Done !
002223.SZ Done !
002690.SZ Done !
600039.SH Done !
600126.SH Done

600563.SH Done !
600166.SH Done !
002707.SZ Done !
601179.SH Done !
600787.SH Done !
603569.SH Done !
600008.SH Done !
600486.SH Done !
601128.SH Done !
600535.SH Done !
300271.SZ Done !
000039.SZ Done !
002681.SZ Done !
000987.SZ Done !
601598.SH Done !
601298.SH Done !
601615.SH Done !
002946.SZ Done !
603379.SH Done !
603317.SH Done !
600968.SH Done !
603256.SH Done !
688088.SH Done !
603983.SH Done !
601456.SH Done !


C:\Users\29186\AppData\Local\Temp\ipykernel_17920\793262935.py:31: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if dt in self.trade_dates:


--------------2019-01-31 为调仓日----------
long_list ['000006.SZ', '000008.SZ', '000025.SZ', '000090.SZ', '000536.SZ', '000587.SZ', '000598.SZ', '000612.SZ', '000636.SZ', '000656.SZ', '000690.SZ', '000712.SZ', '000766.SZ', '000807.SZ', '000829.SZ', '000877.SZ', '000980.SZ', '000999.SZ', '002002.SZ', '002048.SZ', '002051.SZ', '002074.SZ', '002110.SZ', '002127.SZ', '002128.SZ', '002131.SZ', '002152.SZ', '002195.SZ', '002308.SZ', '002358.SZ', '002359.SZ', '002375.SZ', '002400.SZ', '002408.SZ', '002437.SZ', '002463.SZ', '002465.SZ', '002642.SZ', '002707.SZ', '002745.SZ', '002818.SZ', '300001.SZ', '300010.SZ', '300058.SZ', '300113.SZ', '300146.SZ', '300166.SZ', '300266.SZ', '300376.SZ', '300450.SZ', '600006.SH', '600039.SH', '600053.SH', '600056.SH', '600062.SH', '600141.SH', '600151.SH', '600158.SH', '600169.SH', '600259.SH', '600260.SH', '600280.SH', '600366.SH', '600373.SH', '600392.SH', '600393.SH', '600428.SH', '600478.SH', '600500.SH', '600525.SH', '600528.SH', '600582.SH', '600598.SH', 

--------------2019-02-28 为调仓日----------
long_list ['000006.SZ', '000008.SZ', '000025.SZ', '000090.SZ', '000501.SZ', '000528.SZ', '000536.SZ', '000537.SZ', '000587.SZ', '000598.SZ', '000656.SZ', '000690.SZ', '000712.SZ', '000980.SZ', '000998.SZ', '000999.SZ', '002048.SZ', '002051.SZ', '002110.SZ', '002127.SZ', '002131.SZ', '002152.SZ', '002176.SZ', '002221.SZ', '002254.SZ', '002308.SZ', '002354.SZ', '002359.SZ', '002366.SZ', '002372.SZ', '002373.SZ', '002400.SZ', '002408.SZ', '002416.SZ', '002437.SZ', '002439.SZ', '002463.SZ', '002506.SZ', '002642.SZ', '002707.SZ', '002818.SZ', '300001.SZ', '300010.SZ', '300113.SZ', '300146.SZ', '300156.SZ', '300166.SZ', '300266.SZ', '300376.SZ', '300459.SZ', '600039.SH', '600053.SH', '600060.SH', '600141.SH', '600160.SH', '600169.SH', '600171.SH', '600183.SH', '600195.SH', '600260.SH', '600280.SH', '600315.SH', '600366.SH', '600373.SH', '600393.SH', '600428.SH', '600466.SH', '600478.SH', '600525.SH', '600528.SH', '600536.SH', '600582.SH', '600598.SH', 

...

--------------2020-09-30 为调仓日----------
long_list ['000012.SZ', '000021.SZ', '000046.SZ', '000089.SZ', '000301.SZ', '000415.SZ', '000501.SZ', '000543.SZ', '000564.SZ', '000600.SZ', '000681.SZ', '000712.SZ', '000807.SZ', '000869.SZ', '000967.SZ', '000987.SZ', '000990.SZ', '002038.SZ', '002051.SZ', '002064.SZ', '002074.SZ', '002127.SZ', '002138.SZ', '002191.SZ', '002273.SZ', '002353.SZ', '002373.SZ', '002382.SZ', '002390.SZ', '002407.SZ', '002414.SZ', '002444.SZ', '002458.SZ', '002465.SZ', '002491.SZ', '002563.SZ', '002653.SZ', '002709.SZ', '002745.SZ', '002797.SZ', '002815.SZ', '002821.SZ', '002946.SZ', '300002.SZ', '300009.SZ', '300115.SZ', '300133.SZ', '300180.SZ', '300244.SZ', '300257.SZ', '300271.SZ', '300274.SZ', '300357.SZ', '300474.SZ', '600021.SH', '600053.SH', '600062.SH', '600126.SH', '600161.SH', '600167.SH', '600256.SH', '600315.SH', '600329.SH', '600350.SH', '600392.SH', '600415.SH', '600418.SH', '600500.SH', '600515.SH', '600545.SH', '600557.SH', '600572.SH', '600580.SH', 

--------------2020-10-30 为调仓日----------
long_list ['000021.SZ', '000046.SZ', '000061.SZ', '000301.SZ', '000415.SZ', '000501.SZ', '000543.SZ', '000564.SZ', '000600.SZ', '000712.SZ', '000729.SZ', '000813.SZ', '000869.SZ', '000967.SZ', '000987.SZ', '000990.SZ', '002038.SZ', '002051.SZ', '002064.SZ', '002074.SZ', '002078.SZ', '002138.SZ', '002185.SZ', '002191.SZ', '002294.SZ', '002302.SZ', '002373.SZ', '002382.SZ', '002390.SZ', '002414.SZ', '002458.SZ', '002465.SZ', '002491.SZ', '002563.SZ', '002653.SZ', '002709.SZ', '002745.SZ', '002797.SZ', '002815.SZ', '002821.SZ', '002946.SZ', '300002.SZ', '300009.SZ', '300133.SZ', '300251.SZ', '300257.SZ', '300271.SZ', '300296.SZ', '300474.SZ', '300595.SZ', '600021.SH', '600062.SH', '600126.SH', '600167.SH', '600256.SH', '600258.SH', '600329.SH', '600350.SH', '600392.SH', '600415.SH', '600418.SH', '600486.SH', '600497.SH', '600500.SH', '600507.SH', '600515.SH', '600545.SH', '600557.SH', '600566.SH', '600567.SH', '600580.SH', '600645.SH', '600699.SH', 

--------------2020-11-30 为调仓日----------
long_list ['000021.SZ', '000039.SZ', '000046.SZ', '000060.SZ', '000156.SZ', '000400.SZ', '000488.SZ', '000501.SZ', '000543.SZ', '000559.SZ', '000600.SZ', '000712.SZ', '000729.SZ', '000785.SZ', '000813.SZ', '000826.SZ', '000898.SZ', '000932.SZ', '000960.SZ', '000975.SZ', '000990.SZ', '000998.SZ', '001914.SZ', '002038.SZ', '002074.SZ', '002078.SZ', '002127.SZ', '002138.SZ', '002185.SZ', '002217.SZ', '002221.SZ', '002266.SZ', '002285.SZ', '002372.SZ', '002387.SZ', '002390.SZ', '002407.SZ', '002414.SZ', '002444.SZ', '002491.SZ', '002648.SZ', '002653.SZ', '002709.SZ', '002797.SZ', '002839.SZ', '002926.SZ', '002946.SZ', '300001.SZ', '300026.SZ', '300133.SZ', '300166.SZ', '300197.SZ', '300251.SZ', '300271.SZ', '300274.SZ', '300376.SZ', '300474.SZ', '300595.SZ', '600060.SH', '600132.SH', '600160.SH', '600171.SH', '600256.SH', '600260.SH', '600348.SH', '600376.SH', '600418.SH', '600426.SH', '600428.SH', '600486.SH', '600497.SH', '600507.SH', '600515.SH', 

--------------2020-12-31 为调仓日----------
long_list ['000039.SZ', '000046.SZ', '000060.SZ', '000090.SZ', '000156.SZ', '000400.SZ', '000488.SZ', '000501.SZ', '000543.SZ', '000559.SZ', '000581.SZ', '000600.SZ', '000729.SZ', '000813.SZ', '000932.SZ', '000960.SZ', '000990.SZ', '001914.SZ', '002038.SZ', '002064.SZ', '002078.SZ', '002185.SZ', '002217.SZ', '002221.SZ', '002223.SZ', '002233.SZ', '002373.SZ', '002387.SZ', '002390.SZ', '002444.SZ', '002465.SZ', '002603.SZ', '002625.SZ', '002648.SZ', '002653.SZ', '002709.SZ', '002791.SZ', '002797.SZ', '002926.SZ', '002946.SZ', '300001.SZ', '300009.SZ', '300026.SZ', '300133.SZ', '300134.SZ', '300223.SZ', '300257.SZ', '300271.SZ', '300274.SZ', '300296.SZ', '300376.SZ', '300463.SZ', '300474.SZ', '300595.SZ', '600060.SH', '600132.SH', '600256.SH', '600260.SH', '600348.SH', '600376.SH', '600418.SH', '600486.SH', '600497.SH', '600507.SH', '600515.SH', '600517.SH', '600556.SH', '600718.SH', '600733.SH', '600776.SH', '600801.SH', '600827.SH', '600859.SH', 

--------------2021-01-28 为调仓日----------
long_list ['000039.SZ', '000046.SZ', '000060.SZ', '000156.SZ', '000400.SZ', '000559.SZ', '000581.SZ', '000600.SZ', '000729.SZ', '000813.SZ', '000932.SZ', '000937.SZ', '000960.SZ', '000990.SZ', '000998.SZ', '002078.SZ', '002185.SZ', '002195.SZ', '002217.SZ', '002221.SZ', '002233.SZ', '002266.SZ', '002281.SZ', '002372.SZ', '002387.SZ', '002390.SZ', '002399.SZ', '002419.SZ', '002440.SZ', '002444.SZ', '002465.SZ', '002563.SZ', '002648.SZ', '002709.SZ', '002797.SZ', '002815.SZ', '002936.SZ', '300026.SZ', '300072.SZ', '300134.SZ', '300168.SZ', '300223.SZ', '300271.SZ', '300274.SZ', '300296.SZ', '300376.SZ', '600006.SH', '600060.SH', '600079.SH', '600132.SH', '600160.SH', '600256.SH', '600260.SH', '600277.SH', '600339.SH', '600350.SH', '600373.SH', '600376.SH', '600418.SH', '600426.SH', '600435.SH', '600446.SH', '600486.SH', '600497.SH', '600507.SH', '600511.SH', '600515.SH', '600517.SH', '600598.SH', '600718.SH', '600733.SH', '600776.SH', '600801.SH', 

In [17]:
returns.index = pd.to_datetime(returns.index)

In [18]:
pd.DataFrame(returns)

return
index                              
2019-01-02 00:00:00+00:00  0.000000
2019-01-03 00:00:00+00:00  0.000000
2019-01-04 00:00:00+00:00  0.000000
2019-01-07 00:00:00+00:00  0.000000
2019-01-08 00:00:00+00:00  0.000000
...                             ...
2021-01-22 00:00:00+00:00  0.006191
2021-01-25 00:00:00+00:00  0.001407
2021-01-26 00:00:00+00:00 -0.019404
2021-01-27 00:00:00+00:00  0.001648
2021-01-28 00:00:00+00:00 -0.019789

[506 rows x 1 columns]